<a href="https://colab.research.google.com/github/RubenLallave/Bitcoin-project/blob/main/Rub%C3%A8n_Lallave_Kata2_sunlight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# I posit that the number of exposure to daylight hours has a positive influence on median_house_value.
# Let's consider that houses with >=10h of sunlight exposure are "high exposure" and houses with <10h of sunlight exposure are "low exposure"
# You can find the average day length by querying this API https://sunrise-sunset.org/api
# Since getting an average day length may be computationally heavy, let's take the day 01/01/2020 as a reference. Please query the day lenght for this date.

# Hint: don't forget that latitude and longitude parameters to the API have to be strings
# Hint: It may take you a few minutes to query the day_length for all houses.
# Hint: t he day_length comes as a string,but you really only need the "hours" field...

In [26]:
import pandas as pd
import numpy as np
import requests

In [27]:
houses = pd.read_csv('sample_data/california_housing_train.csv').sample(100)
houses.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
11750,-121.31,37.96,52.0,1829.0,301.0,694.0,319.0,3.3466,92600.0
2881,-117.75,34.10,21.0,8069.0,2174.0,4369.0,2036.0,3.2756,156800.0
3311,-117.87,33.86,19.0,2232.0,448.0,1149.0,417.0,3.1534,324400.0
5539,-118.17,34.03,43.0,1636.0,506.0,1855.0,502.0,2.2902,152400.0
8652,-118.54,34.06,21.0,3755.0,525.0,1493.0,526.0,11.4233,500001.0


In [28]:
url = 'https://api.sunrise-sunset.org/json'

params = {
    'lat': 34.86,
    'lng': -120.41,
    'date': '2020-01-01'
}
response = requests.get(url, params=params)
response

<Response [200]>

In [29]:
response.json()

{'results': {'sunrise': '3:07:47 PM',
  'sunset': '1:02:27 AM',
  'solar_noon': '8:05:07 PM',
  'day_length': '09:54:40',
  'civil_twilight_begin': '2:41:17 PM',
  'civil_twilight_end': '1:28:57 AM',
  'nautical_twilight_begin': '2:09:46 PM',
  'nautical_twilight_end': '2:00:28 AM',
  'astronomical_twilight_begin': '1:39:02 PM',
  'astronomical_twilight_end': '2:31:12 AM'},
 'status': 'OK',
 'tzid': 'UTC'}

In [30]:
int(response.json()["results"]["day_length"][:2]) >= 10

False

In [31]:
def function(row):
  url = 'https://api.sunrise-sunset.org/json'
  params = {
      'lat': row["latitude"],
      'lng': row["longitude"],
      'date': '2020-01-01'
      }
  response =requests.get(url, params=params)
  sunlight_hours = int(response.json()["results"]["day_length"][:2])

  if sunlight_hours < 10:
    return "low exposure"
  else:
    return "high exposure"



In [32]:
houses["sun_exposure"] = houses.apply(function, axis=1)

In [33]:
houses

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,sun_exposure
11750,-121.31,37.96,52.0,1829.0,301.0,694.0,319.0,3.3466,92600.0,low exposure
2881,-117.75,34.10,21.0,8069.0,2174.0,4369.0,2036.0,3.2756,156800.0,low exposure
3311,-117.87,33.86,19.0,2232.0,448.0,1149.0,417.0,3.1534,324400.0,low exposure
5539,-118.17,34.03,43.0,1636.0,506.0,1855.0,502.0,2.2902,152400.0,low exposure
8652,-118.54,34.06,21.0,3755.0,525.0,1493.0,526.0,11.4233,500001.0,low exposure
...,...,...,...,...,...,...,...,...,...,...
450,-116.97,33.74,20.0,3674.0,792.0,1498.0,758.0,2.2161,76900.0,high exposure
12584,-121.68,37.98,19.0,3388.0,599.0,1707.0,575.0,3.6411,162800.0,low exposure
15473,-122.31,37.54,46.0,2444.0,397.0,952.0,402.0,4.7500,388200.0,low exposure
1406,-117.18,33.15,7.0,6225.0,1683.0,5410.0,1580.0,2.3200,117500.0,high exposure


In [34]:
houses.groupby("sun_exposure").agg({"median_house_value": ["mean", "count"]})

median_house_value      
                            mean count
sun_exposure                          
high exposure      157907.692308    13
low exposure       227658.678161    87